Google Geocoding

In [ ]:
import pandas as pd
import requests
from geopy.distance import geodesic

df = pd.read_csv("sample1.csv")
API_KEY = "api_key"

def reverse_geocode_google(lat, lon):
    url = f"https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "latlng": f"{lat},{lon}",
        "key": API_KEY
    }
    response = requests.get(url, params=params)
    results = response.json().get("results")
    if results:
        return results[0]["formatted_address"]
    else:
        return "Not found"

def forward_geocode_google(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": API_KEY
    }
    response = requests.get(url, params=params)
    results = response.json().get("results")
    if results:
        location = results[0]["geometry"]["location"]
        return location["lat"], location["lng"]
    else:
        return None, None

def process_row(row):
    try:
        # Reverse geocode
        address = reverse_geocode_google(row['PrvLatitude'], row['PrvLongitude'])

        # Forward geocode
        lat2, lon2 = forward_geocode_google(address)

        if lat2 is not None and lon2 is not None:
            original = (row['PrvLatitude'], row['PrvLongitude'])
            roundtrip = (lat2, lon2)
            distance = geodesic(original, roundtrip).meters
        else:
            lat2, lon2, distance = None, None, None

        return pd.Series([address, lat2, lon2, distance])

    except Exception as e:
        return pd.Series(["Error", None, None, None])

df[['ReverseGeocodedAddress', 'RoundTripLatitude', 'RoundTripLongitude', 'DistanceFromOriginalMeters']] = df.apply(process_row, axis=1)

df.to_csv("sample1_google_geocoded.csv", index=False)
print("Google API round-trip geocoding complete. File saved as 'sample1_google_geocoded.csv'")


✅ Google API round-trip geocoding complete. File saved as 'sample1_google_geocoded.csv'
